# Submission 4
using the entire training set to make the model

In [19]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

import json
import random

In [20]:
data = json.loads(open('data/train.json').read())
# data[:5]

In [23]:
#   replacing spaces with underlines, make into strings for count vectorizer
X_full = ([[ingredient.replace(' ', '_').encode('ascii', 'ignore') for ingredient in food['ingredients']]\
      for food in data])
#  ^Look at the badass list comprehension right there
X_full = [' '.join(ingredients) for ingredients in X_full]

y_full = map(lambda x: x['cuisine'], data)

X_full, y_full = np.asarray(X_full), np.asarray(y_full)

assert len(X_full) == len(y_full)

X_full[66]

'jasmine_rice garlic scallions sugar shiitake Gochujang_base beansprouts top_round_steak sesame_seeds rice_vinegar carrots soy_sauce sesame_oil Taiwanese_bok_choy'

In [25]:
test_data = json.loads(open('data/test.json').read())

X_out_test = ([[ingredient.replace(' ', '_').encode('ascii', 'ignore')\
                for ingredient in food['ingredients']]\
                for food in test_data])
X_out_test = [' '.join(ingredients) for ingredients in X_out_test]
X_out_test = np.asarray(X_out_test)

len(X_out_test)

9944

In [15]:
trans_clf = Pipeline([('count_vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('k_best', SelectKBest(k=5000)),
                      ('clf', SGDClassifier())])

trans_clf.fit(X_full, y_full)
preds = trans_clf.predict(X_out_test)

for a, b in zip(preds[:5], X_out_test[:5]):
    print '{}: {}\n\n'.format(a, b)

british: baking_powder eggs all-purpose_flour raisins milk white_sugar


southern_us: sugar egg_yolks corn_starch cream_of_tartar bananas vanilla_wafers milk vanilla_extract toasted_pecans egg_whites light_rum


italian: sausage_links fennel_bulb fronds olive_oil cuban_peppers onions


cajun_creole: meat_cuts file_powder smoked_sausage okra shrimp andouille_sausage water paprika hot_sauce garlic_cloves browning lump_crab_meat vegetable_oil all-purpose_flour freshly_ground_pepper flat_leaf_parsley boneless_chicken_skinless_thigh dried_thyme white_rice yellow_onion ham


italian: ground_black_pepper salt sausage_casings leeks parmigiano_reggiano_cheese cornmeal water extra-virgin_olive_oil




In [7]:
df = pd.DataFrame(columns=['id','cuisine'])
df['id'] = map(lambda x: x['id'], test_data)

preds = trans_clf.predict(X_out_test)
df['cuisine'] = preds
df.index=df.id
df = df.drop('id', axis=1)
df.head()

,cuisine
id,
18009,irish
28583,southern_us
41580,italian
29752,cajun_creole
35687,italian


In [8]:
df.to_csv('submissions/submission_4.csv')